# Objective: Turning pytorch model into RTL code
## Overview
1. Create a model using pytorch
2. Export a model to ONNX (Open Neural Network Exchange)
3. Convert ONNX → Vitis HLS Project
4. Synthesize HLS C++ → RTL Verilog

In [1]:
ONNX_FILE_NAME = "image_classifier_model.onnx"

### Create a model using pytorch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = (
    torch.accelerator.current_accelerator().type
    if torch.accelerator.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [3]:
class ImageClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = ImageClassifierModel().to(device)
print(model)

ImageClassifierModel(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### Export a model to ONNX (Open Neural Network Exchange)
- reference: https://docs.pytorch.org/tutorials/beginner/onnx/export_simple_model_to_onnx_tutorial.html

In [32]:
torch_model = ImageClassifierModel()
# Create example inputs for exporting the model. The inputs should be a tuple of tensors.
input_tensor = (torch.randn(1, 1, 32, 32),)
torch.onnx.export(
    torch_model,  # model to export
    input_tensor,  # inputs of the model,
    ONNX_FILE_NAME,  # filename of the ONNX model
    input_names=["input"],  # Rename inputs for the ONNX model
    dynamo=False,  # True or False to select the exporter to use
    # keep_initializers_as_inputs=True,
)

/var/folders/0q/g_sh7z092n5gtp0tk43127p40000gn/T/ipykernel_44458/1274601114.py:4: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


### Convert ONNX → Vitis HLS Project
## Reference: 
1. https://fastmachinelearning.org/hls4ml/frontend/qonnx.html
2. https://github.com/fastmachinelearning/hls4ml
3. back end: https://fastmachinelearning.org/hls4ml/backend/vitis.html


In [33]:
import hls4ml
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.channels_last import ConvertToChannelsLastAndClean
from qonnx.transformation.gemm_to_matmul import GemmToMatMul
from qonnx.util.cleanup import cleanup_model

model = ModelWrapper(ONNX_FILE_NAME)
model = cleanup_model(model)
model = model.transform(ConvertToChannelsLastAndClean())
model = model.transform(GemmToMatMul())
model = cleanup_model(model)

config = hls4ml.utils.config.config_from_onnx_model(
    model, granularity="name", backend="Vitis", default_precision="fixed<16,6>"
)
# modify the config as desired
hls_model = hls4ml.converters.convert_from_onnx_model(
    model,
    output_dir="my-hls-test",
    io_type="io_stream",
    backend="Vitis",
    hls_config=config,
)
hls_model.compile()

Output layers:  ['Add_2']
Input shape: [1, 32, 32]
Topology:
Layer name: Transpose_0, layer type: Transpose, current shape: [[1, 1, 32, 32]]
Layer name: Conv_0, layer type: Conv, current shape: [[1, 32, 32, 1], [6, 5, 5, 1], [6]]
Layer name: Relu_0, layer type: Activation, current shape: [[1, 28, 28, 6]]
Layer name: MaxPool_0, layer type: MaxPooling2D, current shape: [[1, 28, 28, 6]]
Layer name: Conv_1, layer type: Conv, current shape: [[1, 14, 14, 6], [16, 5, 5, 6], [16]]
Layer name: Relu_1, layer type: Activation, current shape: [[1, 10, 10, 16]]
Layer name: MaxPool_1, layer type: MaxPooling2D, current shape: [[1, 10, 10, 16]]
Layer name: Flatten_0, layer type: Reshape, current shape: [[1, 5, 5, 16]]
Layer name: MatMul_0, layer type: MatMul, current shape: [[1, 400], [400, 120]]
Layer name: Add_0, layer type: Merge, current shape: [[1, 120], [120]]
Layer name: Relu_2, layer type: Activation, current shape: [[1, 120]]
Layer name: MatMul_1, layer type: MatMul, current shape: [[1, 120],

Exception: Failed to compile project "myproject"